**J'ai essayé de documenter mais je comprends vraiment rien désolé. Je crois que tant que je me serai pas renforcé sur cette histoire de classes et d'objet je capterai rien... :-§**

doc api REST :
http://cataloguekoha.ntrbx.local/api/v1/.html

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import requests
import json

from datetime import datetime
from dateutil.relativedelta import relativedelta

from kiblib.utils.db import DbConn
from kiblib.utils.conf import Config

In [2]:
db_conn = DbConn().create_engine()

webservice_conf = Config().get_config_webservice()
base_url = webservice_conf['base']
username = webservice_conf['user']
password = webservice_conf['pwd']

acquereurs = Config().get_config_stat_sugg()

six_months_date = datetime.today() + relativedelta(months=-6)

In [5]:
url = "http://cataloguekoha.ntrbx.local/api/v1/suggestions/29550"
response = requests.get(url, auth=(username, password))
print(response.json())

{'accepted_by': 8460, 'accepted_date': '2023-03-14', 'archived': False, 'author': 'anouk ricard', 'biblio_id': None, 'budget_id': None, 'collection_title': 'bd rdch', 'copyright_date': None, 'currency': None, 'isbn': '', 'item_price': None, 'item_type': None, 'last_status_change_by': None, 'last_status_change_date': None, 'library_id': 'MED', 'managed_by': 8460, 'managed_date': '2023-03-14', 'note': '(auteur de anna et froga et coucous bouzon)', 'patron_reason': None, 'publication_place': None, 'publication_year': '0', 'publisher_code': '', 'quantity': None, 'reason': None, 'rejected_by': None, 'rejected_date': None, 'status': 'ACCEPTED', 'suggested_by': 36813, 'suggestion_date': '2023-02-22', 'suggestion_id': 29550, 'timestamp': '2023-03-14T15:32:30+01:00', 'title': 'animan', 'total_price': None, 'volume_desc': None}


In [3]:
def delete_suggestion(suggestionid, username, password):
    suggestionid=str(suggestionid)
    url = f"{base_url}/api/v1/suggestions/{suggestionid}"
    response = requests.delete(url, auth=(username, password))
    print(f"{suggestionid} : {response.status_code}")
    
def get_suggestion(suggestionid, username, password):
    suggestionid=str(suggestionid)
    url = f"{base_url}/api/v1/suggestions/{suggestionid}"
    response = requests.get(url, auth=(username, password))
    print(response.json())

def update_suggestion(suggestionid, username, password, data):
    suggestionid=str(suggestionid)
    url = f"{base_url}/api/v1/suggestions/{suggestionid}"
    response = requests.put(url, auth=(username, password), data=data)
    print(response.json())
    
def get_all_suggestions():
    data = []

    url = f"{base_url}/api/v1/suggestions"
    response = requests.get(url, auth=(username, password))
    data.extend(response.json())

    while 'next' in response.links:
        url = response.links['next']['url']
        response = requests.get(url, auth=(username, password))
        data.extend(response.json())
    return data

In [4]:
data = get_all_suggestions()
suggestions = pd.DataFrame(data)
suggestions = suggestions.rename(columns={
    'suggestion_id': 'suggestionid',
    'suggested_by': 'suggestedby',
    'suggestion_date': 'suggesteddate',
    'managed_by': 'managedby',
    'managed_date': 'manageddate',
    'accepted_by': 'acceptedby',
    'accepted_date': 'accepteddate',
    'rejected_by': 'rejectedby',
    'rejected_date': 'rejecteddate',
    'author': 'author',
    'title': 'title',
    'reason': 'reason',
    'patron_reason': 'patronreason',
    'library_id': 'branchcode'
})
suggestions = suggestions[['suggestionid', 'suggestedby', 'suggesteddate', 'managedby',
       'manageddate', 'acceptedby', 'accepteddate', 'rejectedby',
       'rejecteddate', 'status', 'note', 'author', 'title', 'reason', 'patronreason',
       'branchcode']]
len(suggestions)

2133

## Mise à jour des dernières suggestions dans statdb

y aller doucement : en cas d'erreur, on efface toutes les données dans statdb
on faot une sauvegarde au cas où, bien vérifier le nb de lignes à chaque fois.

In [5]:
query = "SELECT * FROM stat_suggestions"
stat_suggestions = pd.read_sql(query, con=db_conn)
# on fait une sauvegarde
stat_suggestions.to_csv(f"../data/stat_suggestions_{datetime.today().strftime('%Y-%m-%d')}.csv.gz", index=False)
len(stat_suggestions)

25271

In [6]:
stat_suggestions = pd.concat([stat_suggestions, suggestions])
len(stat_suggestions)

27404

In [7]:
stat_suggestions = stat_suggestions.drop_duplicates(subset='suggestionid', keep='last')
len(stat_suggestions)

25664

In [9]:
acq_id = stat_suggestions['managedby'].unique()
for c in ['managedby', 'acceptedby', 'rejectedby']:
    for a in acq_id:
        if a in acquereurs:
            stat_suggestions.loc[stat_suggestions[c] == a, c] = acquereurs[a]

stat_suggestions.to_sql('stat_suggestions', con=db_conn, if_exists='replace', index=False)

25664

## On gère les champs dates et on examine la répartition par date

In [10]:
for d in ['suggesteddate', 'manageddate', 'accepteddate', 'rejecteddate']:
    suggestions[d] = pd.to_datetime(suggestions[d])
suggestions['year'] = suggestions['suggesteddate'].dt.year
suggestions['month'] = suggestions['suggesteddate'].dt.month

In [11]:
suggestions.groupby(['status', 'year', 'month'])['suggestionid'].count()

status    year  month
ACCEPTED  2022  12       131
          2023  1        249
                2        194
                3        234
                4        207
                5        139
                6        136
                7         42
ASKED     2023  1          2
                3          2
                5          1
                6          5
                7         40
CHECKED   2023  5          1
                6          3
                7          1
REJECTED  2022  12        67
          2023  1        111
                2         86
                3        162
                4         97
                5         94
                6        101
                7         28
Name: suggestionid, dtype: int64

## Passage des suggestions depuis le statut vérifié vers rejeté

In [12]:
# vérifier toutefosi si c'est bien du rejeté
checked = suggestions[suggestions['status'] == 'CHECKED']
json_object = json.dumps({"status":"REJECTED"})
for suggestionid in checked['suggestionid']:
     update_suggestion(suggestionid, username, password, data=json_object)

{'accepted_by': None, 'accepted_date': None, 'archived': False, 'author': 'Frigiel Piratesourcil', 'biblio_id': None, 'budget_id': None, 'collection_title': '', 'copyright_date': 2023, 'currency': None, 'isbn': '234405572X', 'item_price': None, 'item_type': None, 'last_status_change_by': None, 'last_status_change_date': None, 'library_id': 'MED', 'managed_by': 4446, 'managed_date': '2023-06-27', 'note': '', 'patron_reason': None, 'publication_place': None, 'publication_year': '0', 'publisher_code': ' Glénat', 'quantity': None, 'reason': 'Il y a peu de retours critiques sur ce document et / ou ils sont mitigés.', 'rejected_by': None, 'rejected_date': None, 'status': 'REJECTED', 'suggested_by': 48200, 'suggestion_date': '2023-05-26', 'suggestion_id': 30612, 'timestamp': '2023-07-19T14:21:09+02:00', 'title': 'Mobs - Creeper gaffeur ! MOBS, La vie secrète des monstres Minecraft - Tome 01', 'total_price': None, 'volume_desc': None}
{'accepted_by': None, 'accepted_date': None, 'archived': Fa

## Suppression des anciennes suggestions

In [13]:
sugg2del = suggestions[suggestions['status'] != 'ASKED']
sugg2del = sugg2del[sugg2del['suggesteddate'] <= six_months_date]
print(len(sugg2del))
if len(sugg2del) > 500:
    sugg2del = sugg2del[0:500] # attention, procéder par lots réduits
    print(f"Attention, procéder en plusieurs fois")

464


In [14]:
sugg2del['month'] = sugg2del['suggesteddate'].dt.month
sugg2del.groupby(['status', 'year', 'month'])['suggestionid'].count()

status    year  month
ACCEPTED  2022  12       131
          2023  1        197
REJECTED  2022  12        67
          2023  1         69
Name: suggestionid, dtype: int64

In [15]:
for suggestionid in sugg2del['suggestionid']:
    delete_suggestion(suggestionid, username=username, password=password)

28664 : 204
28665 : 204
28666 : 204
28667 : 204
28668 : 204
28669 : 204
28670 : 204
28671 : 204
28672 : 204
28673 : 204
28674 : 204
28675 : 204
28676 : 204
28677 : 204
28678 : 204
28680 : 204
28681 : 204
28683 : 204
28684 : 204
28685 : 204
28686 : 204
28687 : 204
28688 : 204
28689 : 204
28690 : 204
28691 : 204
28692 : 204
28693 : 204
28695 : 204
28696 : 204
28698 : 204
28699 : 204
28700 : 204
28701 : 204
28702 : 204
28703 : 204
28704 : 204
28706 : 204
28707 : 204
28708 : 204
28709 : 204
28710 : 204
28711 : 204
28712 : 204
28713 : 204
28714 : 204
28715 : 204
28716 : 204
28717 : 204
28718 : 204
28719 : 204
28720 : 204
28721 : 204
28722 : 204
28723 : 204
28724 : 204
28725 : 204
28726 : 204
28727 : 204
28728 : 204
28729 : 204
28730 : 204
28731 : 204
28734 : 204
28735 : 204
28736 : 204
28738 : 204
28739 : 204
28741 : 204
28742 : 204
28743 : 204
28744 : 204
28746 : 204
28748 : 204
28749 : 204
28750 : 204
28752 : 204
28753 : 204
28754 : 204
28755 : 204
28756 : 204
28757 : 204
28758 : 204
2875

## Statistiques sur les suggestions

In [16]:
query = "SELECT * FROM stat_suggestions WHERE YEAR(suggesteddate) >= 2015 "
stat_sugg = pd.read_sql(query, con=db_conn)
stat_sugg['suggesteddate'] = pd.to_datetime(stat_sugg['suggesteddate'])
stat_sugg['year'] = stat_sugg['suggesteddate'].dt.year
stat_sugg['month'] = stat_sugg['suggesteddate'].dt.month
stat_sugg['week'] = stat_sugg['suggesteddate'].dt.isocalendar().week

In [17]:
stat_sugg.groupby('year').agg({'suggestionid': 'count', 'suggestedby': 'nunique'}).reset_index()

,year,suggestionid,suggestedby
0,2015,1146,343
1,2016,2543,503
2,2017,2911,547
3,2018,3467,630
4,2019,3564,656
5,2020,2948,524
6,2021,3373,569
7,2022,3455,640
8,2023,1989,451


In [98]:
biblionumber = 35670
url = f"{base_url}/api/v1/biblios/{biblionumber}"
response = requests.get(url, auth=(username, password))
response

<Response [406]>